# Day 14: Extended Polymerization

[*Advent of Code 2021 day 14*](https://adventofcode.com/2021/day/14) and [*solution megathread*](https://redd.it/rfzq6f)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2021/14/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2021%2F14%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys

sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [2]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

## Comments

This time I won't ignore the complexity hint - and I see little reason a bigram approach wouldn't work. Let's go.

In [5]:
testdata = []
testdata.append(("""NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C""".splitlines()))

inputdata = downloaded['input'].splitlines()

In [6]:
testdata[0]

['NNCB',
 '',
 'CH -> B',
 'HH -> N',
 'CB -> H',
 'NH -> C',
 'HB -> C',
 'HC -> B',
 'HN -> C',
 'NN -> C',
 'BH -> H',
 'NC -> B',
 'NB -> B',
 'BN -> B',
 'BB -> N',
 'BC -> B',
 'CC -> N',
 'CN -> C']

In [7]:
from collections.abc import Iterable


def parse_data(data: list[str]) -> \
        tuple[str, dict[str, str]]:
    result_dict = dict()
    for line in data[2:]:
        k, v = line.split(' -> ', 2)
        result_dict[k] = v
    return data[0], result_dict


def defaultdict(d: dict, k: str, v: int = 0):
    if not v:
        v = 1
    if k not in d:
        d[k] = v
    else:
        d[k] += v


def bigrams(template: str) -> dict[str, int]:
    assert(len(template) > 3)
    result = dict()
    for c1, c2 in zip(template[:-1], template[1:]):
        bg = c1 + c2
        defaultdict(result, bg)
    return result


def apply_poly(bgs: dict[str, int],
               poly: dict[str, str],
               last_bg: str) -> tuple[dict[str, int], str]:
    result = dict()
    for bg, c in bgs.items():
        if bg not in poly:
            defaultdict(result, bg, c)
        else:
            if bg == last_bg:
                last_bg = poly[bg] + bg[1]
            p1, p2 = bg[0] + poly[bg], poly[bg] + bg[1]
            defaultdict(result, p1, c)
            defaultdict(result, p2, c)
    return result, last_bg


def count_chars(bgs: dict[str, int], last_bg: str) -> dict[str, int]:
    result = dict()
    for k, v in bgs.items():
        defaultdict(result, k[0], v)
    defaultdict(result, last_bg[1])
    return result


def my_part1_solution(data: str,
                      debug: bool = False) -> int:
    template, poly = parse_data(data)
    bgs = bigrams(template)
    last_bg = template[-2:]
    for i in range(1, 10 + 1):
        bgs, last_bg = apply_poly(bgs, poly, last_bg)
        if debug:
            print(f'After cycle {i}: {bgs}')
    elements = count_chars(bgs, last_bg)
    if debug:
        print(f'Resulting elements: {elements}')
    return max(elements.values()) - min(elements.values())

In [8]:
my_part1_solution(testdata[0], debug=True)
# assert(my_part1_solution(testdata[0][0], debug=True)
#     == testdata[0][1])

After cycle 1: {'NC': 1, 'CN': 1, 'NB': 1, 'BC': 1, 'CH': 1, 'HB': 1}
After cycle 2: {'NB': 2, 'BC': 2, 'CC': 1, 'CN': 1, 'BB': 2, 'CB': 2, 'BH': 1, 'HC': 1}
After cycle 3: {'NB': 4, 'BB': 4, 'BC': 3, 'CN': 2, 'NC': 1, 'CC': 1, 'BN': 2, 'CH': 2, 'HB': 3, 'BH': 1, 'HH': 1}
After cycle 4: {'NB': 9, 'BB': 9, 'BN': 6, 'BC': 4, 'CC': 2, 'CN': 3, 'NC': 1, 'CB': 5, 'BH': 3, 'HC': 3, 'HH': 1, 'HN': 1, 'NH': 1}
After cycle 5: {'NB': 19, 'BB': 19, 'BN': 15, 'BC': 8, 'CN': 6, 'NC': 3, 'CC': 3, 'CH': 6, 'HB': 8, 'BH': 3, 'HH': 3, 'HN': 1, 'NH': 1, 'HC': 1}
After cycle 6: {'NB': 41, 'BB': 42, 'BN': 34, 'BC': 12, 'CC': 6, 'CN': 10, 'NC': 4, 'CB': 14, 'BH': 9, 'HC': 9, 'HH': 3, 'HN': 3, 'NH': 3, 'CH': 1, 'HB': 1}
After cycle 7: {'NB': 87, 'BB': 87, 'BN': 76, 'BC': 25, 'CN': 19, 'NC': 9, 'CC': 10, 'CH': 17, 'HB': 23, 'BH': 10, 'HH': 9, 'HN': 3, 'NH': 3, 'HC': 4, 'CB': 2}
After cycle 8: {'NB': 183, 'BB': 188, 'BN': 163, 'BC': 38, 'CC': 19, 'CN': 32, 'NC': 13, 'CB': 40, 'BH': 27, 'HC': 26, 'HH': 10, 'HN

1588

In [9]:
my_part1_solution(inputdata)

4517

In [10]:
HTML(downloaded['part1_footer'])

## Part Two

In [11]:
HTML(downloaded['part2'])

In [12]:
def my_part2_solution(data: str,
                      debug: bool = False) -> int:
    template, poly = parse_data(data)
    bgs = bigrams(template)
    last_bg = template[-2:]
    for i in range(1, 40 + 1):
        bgs, last_bg = apply_poly(bgs, poly, last_bg)
        if debug:
            print(f'After cycle {i}: {bgs}')
    elements = count_chars(bgs, last_bg)
    if debug:
        print(f'Resulting elements: {elements}')
    return max(elements.values()) - min(elements.values())

In [13]:
my_part2_solution(testdata[0])

2188189693529

In [14]:
my_part2_solution(inputdata)

4704817645083

In [15]:
HTML(downloaded['part2_footer'])